# Graph three-colorability

In this chapter we construct a zero-knowledge protocol around graph colorability.

This chaper was inspired by [a video by Numberphile](https://www.youtube.com/watch?v=5ovdoxnfFVc) and is based on [a lecture from the Max Plack Institute for Informatics](https://resources.mpi-inf.mpg.de/departments/d1/teaching/ss13/gitcs/lecture9.pdf).

# What is a graph?

[A graph](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics)) consists of nodes and edges. Nodes are points in space. Edges are bridges between nodes.

# What is a (three-)coloring?

**Coloring** a graph means to assign a color to each node. For each edge, adjacent nodes must have a different color.

A coloring comes with a number of colors. How many colors we have available determines how hard it is to color a graph.

For instance, [it was proven that every graph can be four-colored](https://en.wikipedia.org/wiki/Four_color_theorem). In contrast, not every graph can be three-colored! In fact, three-coloring is an NP-complete problem. This means that for large graphs it may take exponential time to three-color them.

A graph is **three-colorable** if there is a three-coloring for it.

# What are we proving?

Peggy and Victor are engaged in an interactive proof.

There is a graph.

Peggy thinks she knows a three-coloring of the graph. She wants to prove that to Victor, without revealing the coloring.

Victor is sceptical and wants to see evidence. He wants to expose Peggy as a liar if her coloring is invalid.

Peggy wins if she convinces Victor. Victor wins by accepting only valid three-colorings.

# Jupyter setup

Run the following snippet to set up your Jupyter notebook for the workshop.

In [ ]:
import os
import sys

# Add project root so we can import local modules
root_dir = sys.path.append("..")
sys.path.append(root_dir)

# Import here so cells don't depend on each other
from IPython.display import display
from typing import List, Tuple, Dict
import ipywidgets as widgets
import random
import networkx as nx
import matplotlib.pyplot as plt

from local.graph import Mapping, three_colorable_graph, not_three_colorable_graph
from local.ec.static import CurvePoint, Scalar, ONE_POINT
from local.ec.util import Opening
import local.stats as stats

# Select the scenario

Choose the good or the evil scenario. See how it affects the other cells further down.

1. **Peggy is honest** 😇 She knows a three-coloring of the graph. She wants to convince Victor of a true statement.
2. **Peggy is lying**  😈 She doesn't know a three-coloring of the graph! She tries to fool Victor into believing a false statement.

Also select the **size of the graphs**.

In [ ]:
def generate_coloring(values: Dict):
    global graph, coloring

    if honest_dropdown.value:
        # Good: Valid three-coloring
        graph, coloring = three_colorable_graph(n_nodes_slider.value)
    else:
        # Evil: Invalid three-coloring
        graph, coloring = not_three_colorable_graph(n_nodes_slider.value)

honest_dropdown = widgets.Dropdown(
    options=[
        ("Peggy can three-color 😇", True),
        ("Peggy cannot three-color 😈", False)],
    value=True,
    description="Scenario:",
)
honest_dropdown.observe(generate_coloring, names="value")

n_nodes_slider = widgets.IntSlider(min=4, max=20, value=4, step=1, description="#Nodes")
n_nodes_slider.observe(generate_coloring, names="value")

# Generate default values
generate_coloring({})
# Display selection
display(honest_dropdown)
display(n_nodes_slider)

# Visualize your graph

Visualize the graph you generated.

You will see the colors in the plot.

In [ ]:
node_colors = [coloring[node] for node in graph.nodes]
nx.draw(graph, node_color=node_colors, with_labels=True)
plt.show()

# How the proof goes

Victor knows the graph. Peggy knows a three-coloring of the graph.

1. Peggy randomly shuffles her colors.
1. Peggy sends commitments to the (shuffled) color of each node to Victor.
1. Victor selects a random edge.
1. Peggy opens the commitments of the colors of both nodes of the edge.
1. Victor checks if the colors are different.

# Why Peggy shuffles colors

Step 1 prevents Victor from knowing Peggy's coloring. Victor sees only one edge which is not enough information to derive the entire coloring. Peggy shuffles differently each time, so Victor cannot learn across multiple rounds.

Each three-coloring gives rise to six variants where the colors are shuffled around. You see the original colors (header) and the possible variants (lines) in the table below.

| red   | blue  | green |
|-------|-------|-------|
| red   | blue  | green |
| red   | green | blue  |
| blue  | red   | green |
| blue  | green | red   |
| green | red   | blue  |
| green | blue  | red   |

# Why Peggy commits to all colors

Peggy commits to the (shuffled) color of each edge. If there is an invalid edge where both nodes have the same color, then Victor has a chance of randomly selecting this edge. Peggy is forced to open the commitment to the correct value, which she sends to Victor. He sees that the colors are the same and rejects her coloring proof.

In [ ]:
punto_uno, = CurvePoint.sample_greater_one(1)

Edge = Tuple[int, int]
ColoredEdge = Tuple[Opening, Opening]

class Peggy:
    def __init__(self, coloring: Dict[int, int]):
        self.coloring = Mapping(coloring)
        
    def commit(self) -> List[CurvePoint]:
        shuffle = Mapping.shuffle_list([0, 1, 2])
        shuffled_coloring = self.coloring.and_then(shuffle)
        
        self.openings = [Opening(Scalar(shuffled_coloring[index]), ONE_POINT, punto_uno)
                         for index in self.coloring]
        commitments = [opening.close() for opening in self.openings]
        
        return commitments
    
    def color_edge(self, edge: Edge) -> ColoredEdge:
        return self.openings[edge[0]], self.openings[edge[1]]


class Victor:
    def __init__(self, graph: nx.Graph):
        self.edges = sorted(graph.edges())

    def random_edge(self, commitments: List[CurvePoint]) -> Edge:
        self.commitments = commitments
        self.edge = random.choice(self.edges)
        
        return self.edge
    
    def verify(self, colored_edge: ColoredEdge) -> bool:
        # Invalid coloring
        if colored_edge[0].value() == colored_edge[1].value():
            return False
        # Openened colors correspond to commitments
        if not colored_edge[0].verify(self.commitments[self.edge[0]]):
            return False
        if not colored_edge[1].verify(self.commitments[self.edge[1]]):
            return False
        
        return True

# Run the proof

Let's see the proof in action.

Run the Python code below and see what happens.

The outcome depends on the scenario you picked. The outcome is also randomly different each time.

Feel free to run the code multiple times!

In [ ]:
peggy = Peggy(coloring)
victor = Victor(graph)

commitments = peggy.commit()
print("Node color commitments: {}".format(commitments))

edge = victor.random_edge(commitments)
print("Edge: {}".format(edge))

colored_edge = peggy.color_edge(edge)
print("Colored edge: {}".format(colored_edge))
print()

if victor.verify(colored_edge):
    if honest_dropdown.value:
        print("Victor is convinced 👌 (expected)")
    else:
        print("Victor is convinced 👌 (Victor was fooled)")
else:
    if honest_dropdown.value:
        print("Victor is not convinced... 🤨 (Peggy was dumb)")
    else:
        print("Victor is not convinced... 🤨 (expected)")

# How the proof is complete

If Peggy can three-color the graph, then **Victor will always be convinced** by her proof.

This is because Peggy commits to a correct three-coloring. The edge that Victor chooses is always correctly colored.

Let's run a couple of exchanges and see how they go.

In [ ]:
n_exchanges_complete_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_exchanges_complete_slider

In [ ]:
# Good scenario:
# Peggy knows a valid three-coloring
graph2, coloring2 = three_colorable_graph(n_nodes_slider.value)

honest_peggy = Peggy(coloring2)
victor = Victor(graph2)

peggy_success = 0

for _ in range(n_exchanges_complete_slider.value):
    commitments = honest_peggy.commit()
    edge = victor.random_edge(commitments)
    colored_edge = honest_peggy.color_edge(edge)

    if victor.verify(colored_edge):
        peggy_success += 1
        
peggy_success_rate = peggy_success / n_exchanges_complete_slider.value * 100

print(f"Running {n_exchanges_complete_slider.value} exchanges.")
print(f"Honest Peggy wins {peggy_success_rate:0.2f}% of the time.")
print()

assert peggy_success_rate == 100
print("Peggy always wins if she is honest.")

# How the proof is sound

If Peggy cannot three-color the graph, then **Victor has a chance to reject** her proof.

If $u$ out of $v$ edges are correctly colored, then Victor has a $\frac{u}{v}$ chance of randomly selecting one of these edges. In this case, Peggy can reveal two different colors and pass Victor's test.

The more edges are incorrect, the likelier it becomes for Victor to select an incorrect edge. Still, these probabilities don't look good for Victor.

We can increase Victor's confidence by running the protocol for **multiple rounds**.

This means Peggy randomly shuffles her colors and Victor randomly selects an edge multiple times. Each time, Peggy has to reveal the colors of the selected edge. Victor accepts if Peggy answered correctly **all** time times. However, he rejects if Peggy answers incorrectly **even once**.

The chance that Peggy answers correctly for $n$ rounds, while $u$ out of $v$ edges are correctly colored, is $\left(\frac{u}{v}\right)^n$. Because $u < v$, this decreases exponentially in $n$ and becomes tiny! If Peggy answers correctly, then Victor is confident that she didn't cheat.

Let's run a couple of exchanges and see how they go.

In [ ]:
n_exchanges_sound_slider = widgets.IntSlider(min=10, max=1000, value=10, step=10, description="#Exchanges")
n_rounds_slider = widgets.IntSlider(min=1, max=15, value=1, step=1, description="#Rounds")

display(n_exchanges_sound_slider)
display(n_rounds_slider)

In [ ]:
# Evil scenario:
# Peggy doesn't know a valid three-coloring
graph3, coloring3 = not_three_colorable_graph(n_nodes_slider.value)

lying_peggy = Peggy(coloring3)
victor = Victor(graph3)

victor_success = 0

for _ in range(n_exchanges_sound_slider.value):
    for _ in range(n_rounds_slider.value):
        commitments = lying_peggy.commit()
        edge = victor.random_edge(commitments)
        colored_edge = lying_peggy.color_edge(edge)
    
        if not victor.verify(colored_edge):
            victor_success += 1
            break
            
victor_success_rate = victor_success / n_exchanges_sound_slider.value * 100

print(f"Running {n_exchanges_sound_slider.value} exchanges with {n_rounds_slider.value} rounds each.")
print(f"Victor wins against lying Peggy {victor_success_rate:0.2f}% of the time.")
print()

if victor_success_rate < 50:
    print("Victor loses quite often for a small number of rounds.")
elif victor_success_rate < 90:
    print("Victor gains more confidence with each added round.")
else:
    print("At some point it is basically impossible to fool Victor.")

# How the proof is zero-knowledge

The proof itself looks like random noise. Nothing can be extracted from this noise.

Everything that is sent over the wire is randomized:

1. Peggy sends commitments which look like random points.
1. Victor randomly selects an edge.
1. Peggy opens the commitments to the selected edge. The commitments contain a randomly shuffled color and a random blinding factor.

We can replicate this pattern:

1. Select a random edge.
1. Select a random left color.
1. Select a random right color that is different from the left color.
1. Create commitments for the left and right node.
1. Create commitments to random colors for the remaining nodes.

In the fake transcripts, we select the edge first. Then, we color it. Then, we create the commitments.

By construction, the colors of the selected edge are different.

Let's run a chi-square test to see if the original transcripts are distinguishable from the fake transcripts.

**Try small graphs first!** They require fewer samples than large graphs.

Because of the large number of commitments, **this experiment requires especially many samples (#transcripts)**. Make sure to use small graphs. The code also cheats by making the commitment openings more compact than they actually are.

In [ ]:
n_transcripts_slider = widgets.IntSlider(min=1000, max=100000, value=10000, step=1000, description="#Transcripts")
n_transcripts_slider

In [ ]:
peggy = Peggy(coloring)
victor = Victor(graph)
edges = sorted(graph.edges())  # for fake transcripts

def real_transcript() -> Tuple:
    commitments = peggy.commit()
    edge = victor.random_edge(commitments)
    left_open, right_open = peggy.color_edge(edge)

    return CurvePoint.batch_serialize(commitments, compact=2), edge, \
           Opening.batch_serialize((left_open, right_open), compact=2)


def fake_transcript() -> Tuple:
    edge = random.choice(edges)
    left_node, right_node = edge
    
    left_color = random.randrange(0, 3)
    if random.random() > 0.5:
        right_color = (left_color + 1) % 3
    else:
        right_color = (left_color - 1) % 3
        
    left_color = Scalar(left_color)
    right_color = Scalar(right_color)
    
    left_open = Opening(left_color, ONE_POINT, punto_uno)
    right_open = Opening(right_color, ONE_POINT, punto_uno)
    commitments = [CurvePoint.random() for _ in range(len(graph.nodes))]
    commitments[left_node] = left_open.close()
    commitments[right_node] = right_open.close()
    
    return CurvePoint.batch_serialize(commitments, compact=2), edge, \
           Opening.batch_serialize((left_open, right_open), compact=2)

print("Real transcript: {}".format(real_transcript()))
print("Fake transcript: {}".format(fake_transcript()))
print()

real_samples = [real_transcript() for _ in range(n_transcripts_slider.value)]
fake_samples = [fake_transcript() for _ in range(n_transcripts_slider.value)]

null_hypothesis = stats.chi_square_equal(real_samples, fake_samples)
print()

if null_hypothesis:
    print("Real and fake transcripts are the same distribution.")
    print("Victor learns nothing 👌")
else:
    print("Real and fake transcripts are different distributions.")
    print("Victor might learn something 😧")

stats.plot_comparison(real_samples, fake_samples, "real", "fake")